# LSTM on Recipe Data

**The notebook has been adapted from the notebook provided in David Foster's Generative Deep Learning, 2nd Edition.**

- Book: [Amazon](https://www.amazon.com/Generative-Deep-Learning-Teaching-Machines/dp/1098134184/ref=sr_1_1?keywords=generative+deep+learning%2C+2nd+edition&qid=1684708209&sprefix=generative+de%2Caps%2C93&sr=8-1)
- Original notebook (tensorflow and keras): [Github](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/05_autoregressive/01_lstm/lstm.ipynb)
- Dataset: [Kaggle](https://www.kaggle.com/datasets/hugodarwood/epirecipes)

In [2]:
import re
import string
import json
from collections import defaultdict
import time

import numpy as np
import jax
import jax.numpy as jnp
from tensorflow.data import Dataset
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import utils

from flax import struct
from flax.training import train_state
import flax.linen as nn
import optax

from clu import metrics

## 0. Train parameters

In [3]:
DATA_DIR = '../../data/epirecipes/full_format_recipes.json'

EMBEDDING_DIM = 100
HIDDEN_DIM = 128
NUM_LSTM_LAYERS = 1
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 32
EPOCHS = 30
VOCAB_SIZE = 8200
LR = 1e-3

MAX_PAD_LEN = 200
MAX_VAL_TOKENS = 100 # Max number of tokens when generating texts

## 1. Load dataset

In [4]:
def pad_punctuation(sentence):
    sentence = re.sub(f'([{string.punctuation}])', r' \1 ', sentence)
    sentence = re.sub(' +', ' ', sentence)
    return sentence

In [5]:
# load dataset
with open(DATA_DIR, 'r+') as f:
    recipe_data = json.load(f)

In [6]:
# preprocess dataset
filtered_data = [
    'Recipe for ' + x['title'] + ' | ' + ' '.join(x['directions'])
    for x in recipe_data
    if 'title' in x and x['title']
    and 'directions' in x and x['directions']
]

text_ds = [pad_punctuation(sentence) for sentence in filtered_data]
print(f'Total recipe loaded: {len(text_ds)}')

Total recipe loaded: 20098


In [7]:
print('Sample data:')
sample_data = np.random.choice(text_ds)
print(sample_data)

Sample data:
Recipe for Buttermilk Spoon Bread | Preheat oven to 350°F . Generously butter 13x9x2 - inch glass baking dish . Whisk eggs and cream in large bowl to blend . Bring 4 cups water , buttermilk , butter , salt , and pepper to boil in heavy large saucepan over medium - high heat . Gradually whisk in cornmeal . Reduce heat to medium - low . Simmer until thick and smooth , whisking frequently , about 4 minutes . Gradually whisk hot cornmeal into egg mixture . Transfer batter to prepared dish . Bake uncovered until golden on top and gently set in center , about 30 minutes . Let stand 10 minutes . Serve warm , directly from dish . 


## 2. Build vocabularies

In [8]:
# conver texts list to tf dataset
text_ds_tf = Dataset.from_tensor_slices(text_ds)

vectorize_layer = TextVectorization(
    standardize='lower',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_PAD_LEN+1
)

In [9]:
vectorize_layer.adapt(text_ds_tf)
vocab = vectorize_layer.get_vocabulary()
index_to_word = {index : word for index, word in enumerate(vocab)}
word_to_index = {word : index for index, word in enumerate(vocab)}

# First 10 items in the vocabulary
for i, word in enumerate(vocab[:10]):
    print(f'{i}: {word}')

0: 
1: [UNK]
2: .
3: ,
4: and
5: to
6: in
7: the
8: with
9: a


In [10]:
sample_data_tokenized = vectorize_layer(sample_data)
print('Source text:')
print(sample_data)
print('\n')
print('Mapped sample:')
print(sample_data_tokenized.numpy())

Source text:
Recipe for Buttermilk Spoon Bread | Preheat oven to 350°F . Generously butter 13x9x2 - inch glass baking dish . Whisk eggs and cream in large bowl to blend . Bring 4 cups water , buttermilk , butter , salt , and pepper to boil in heavy large saucepan over medium - high heat . Gradually whisk in cornmeal . Reduce heat to medium - low . Simmer until thick and smooth , whisking frequently , about 4 minutes . Gradually whisk hot cornmeal into egg mixture . Transfer batter to prepared dish . Bake uncovered until golden on top and gently set in center , about 30 minutes . Let stand 10 minutes . Serve warm , directly from dish . 


Mapped sample:
[  26   16  599   98  195   27   86   47    5  216    2  540   50 1010
   13   53  287   58  155    2   73  199    4   76    6   30   21    5
  139    2   84   32  138   39    3  599    3   50    3   24    3    4
   33    5   69    6   78   30   80   20   29   13   75   17    2  365
   73    6  698    2  153   17    5   29   13  134    2

## 3. Create train/validation datasets

In [11]:
# map a single text slice into source and targets
def map_src_tgt(text):
    tokenized_sentence = vectorize_layer(text)
    src = tokenized_sentence[:-1]
    tgt = tokenized_sentence[1:]
    return src, tgt
    
# create datasets
def get_datasets(input_ds):
    train_size = int(len(input_ds) * (1 - VALIDATION_SPLIT))
    train_ds = input_ds.take(train_size) # take train dataset
    valid_ds = input_ds.skip(train_size) # take validation dataset
    print(f'train size: {train_ds.cardinality()}, valid size: {valid_ds.cardinality()}')

    train_ds = train_ds.map(map_src_tgt)
    valid_ds = valid_ds.map(map_src_tgt)
    
    train_ds = train_ds.batch(BATCH_SIZE).shuffle(1024).prefetch(1)
    valid_ds = valid_ds.batch(BATCH_SIZE).prefetch(1)

    print(f'train batch: {train_ds.cardinality()}, valid batch: {valid_ds.cardinality()}')
    return train_ds, valid_ds

In [12]:
train_ds, valid_ds = get_datasets(text_ds_tf)

train size: 16078, valid size: 4020
train batch: 503, valid batch: 126


## 4. Build LSTM model

In [13]:
class LSTM_model(nn.Module):

    num_lstm_layers: int
    
    def setup(self):
        self.embed = nn.Embed(num_embeddings=VOCAB_SIZE, features=HIDDEN_DIM)

        
        self.lstm_layers = [nn.RNN(nn.OptimizedLSTMCell(), HIDDEN_DIM) 
                                for _ in range(self.num_lstm_layers)]
        
        self.dense = nn.Dense(features=VOCAB_SIZE)

    def __call__(self, x):
        # Embedding
        x = self.embed(x)

        # carry, hidden = nn.OptimizedLSTMCell.initialize_carry(jax.random.PRNGKey(0),
        #                                                       batch_dims=(len(x),),
        #                                                       size=HIDDEN_DIM)
        # (carry, hidden), x = self.lstm((carry, hidden), x)
        # variables = self.lstm.init(jax.random.PRNGKey(0), x)
        for lstm in self.lstm_layers:
            # x = self.lstm(x)
            x = lstm(x)

        # # LSTM
        # for i in range(self.num_lstm_layers):
        #     x = self.lstm_layers[i](x)
        #     # hidden_state = self.lstm_layers[i].initialize_carry(jax.random.PRNGKey(0), (x.shape[0], x.shape[1]), HIDDEN_DIM)
        #     # _, x = self.lstm_layers[i](hidden_state, x)
        
        # Dense layer
        x = self.dense(x)
        return x

lstm_model = LSTM_model(NUM_LSTM_LAYERS)
rng = jax.random.PRNGKey(0)

print(lstm_model.tabulate(rng, jnp.ones((BATCH_SIZE, MAX_PAD_LEN), dtype=int), depth=1))


                                            LSTM_model Summary                                            
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path          ┃ module     ┃ inputs              ┃ outputs              ┃ params                       ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│               │ LSTM_model │ int32[32,200]       │ float32[32,200,8200] │                              │
├───────────────┼────────────┼─────────────────────┼──────────────────────┼──────────────────────────────┤
│ embed         │ Embed      │ int32[32,200]       │ float32[32,200,128]  │ embedding: float32[8200,128] │
│               │            │                     │                      │                              │
│               │            │                     │                      │ 1,049,600 (4.2 MB)           │
├───────────────┼────────────┼──────

## 5. Create `TrainState`

In [22]:
@struct.dataclass
class Metrics(metrics.Collection):
    loss: metrics.Average.from_output('loss')

class TrainState(train_state.TrainState):
    metrics: Metrics

# train state for lstm model
def create_train_state(model, param_key, learning_rate):
    # initialize model
    params = model.init(param_key, jnp.ones((BATCH_SIZE, MAX_PAD_LEN), dtype=int))['params']
    # initialize optimizer
    tx = optax.adam(learning_rate=learning_rate)
    return TrainState.create(
            apply_fn=model.apply,
            params=params,
            tx=tx,
            metrics=Metrics.empty())

## 6. Train step functions

In [23]:
# train step
@jax.jit
def train_step(state, batch):
    def loss_fn(params):
        preds = state.apply_fn({'params': params}, batch[0])
        loss = optax.softmax_cross_entropy_with_integer_labels(preds, batch[1]).mean()
        return loss

    # compute loss and apply gradients
    grad_fn = jax.value_and_grad(loss_fn)
    loss, grads = grad_fn(state.params)
    state = state.apply_gradients(grads=grads)

    # Update metrics
    metric_updates = state.metrics.single_from_model_output(loss=loss)
    metrics = state.metrics.merge(metric_updates)
    state = state.replace(metrics=metrics)
    return state 


# evaluation
@jax.jit
def validation(state, batch):
    preds = state.apply_fn({'params': state.params}, batch[0])
    loss = optax.softmax_cross_entropy_with_integer_labels(preds, batch[1]).mean()

    # Update metrics
    metric_updates = state.metrics.single_from_model_output(loss=loss)
    metrics = state.metrics.merge(metric_updates)
    state = state.replace(metrics=metrics)
    return state

In [24]:
# get next-word probability distribution
@jax.jit
def get_probs(state, input_tokens):
    return state.apply_fn({'params': state.params}, input_tokens)[0][-1]


# Text generator
class TextGenerator():
    def __init__(self, index_to_word):
        self.index_to_word = index_to_word
        self.word_to_index = {word : index for index, word in index_to_word.items()}

    # scaling the model's output probability with temperature
    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(VOCAB_SIZE, p=probs), probs
    
    # generate text
    def generate(self, state, start_prompt, max_tokens, temperature, output_info=False):
        
        start_tokens = [self.word_to_index[word] for word in start_prompt.split()]
        sample_token = None
        info = []

        while len(start_tokens) < max_tokens and sample_token != 0:
            input_tokens = np.array(start_tokens).reshape(1, -1)
            probs = get_probs(state, input_tokens)
            probs = nn.log_softmax(probs)
            sample_token, probs = self.sample_from(np.exp(probs), temperature)
            start_tokens.append(sample_token)
            if output_info:
                info.append({'tokens': np.copy(start_tokens), 'word_probs': probs})
            
        output_text = [self.index_to_word[token] for token in start_tokens if token != 0]
        print(' '.join(output_text))

        return info

## 7. Training

In [25]:
lstm_model = LSTM_model(NUM_LSTM_LAYERS)
state = create_train_state(lstm_model, jax.random.PRNGKey(0), learning_rate=LR)
text_generator = TextGenerator(index_to_word)

In [26]:
loss_hist = defaultdict(list)

for i in range(EPOCHS):
    prev_time = time.time()
    
    #training
    for batch in train_ds.as_numpy_iterator():
        state = train_step(state, batch)

    train_loss = state.metrics.compute()['loss']
    state = state.replace(metrics=state.metrics.empty())

    #validation
    test_state = state
    for batch in valid_ds.as_numpy_iterator():
        test_state = validation(test_state, batch)

    valid_loss = test_state.metrics.compute()['loss']
    
    loss_hist['train_loss'].append(train_loss)
    loss_hist['valid_loss'].append(valid_loss)

    curr_time = time.time()
    print(f'Epoch: {i+1}\tepoch time {(curr_time - prev_time) / 60:.2f} min')
    print(f'\ttrain loss: {train_loss:.4f}, valid loss: {valid_loss:.4f}')
    
    if (i + 1) % 10 == 0:
        # generate text
        print('\nGenerated text:')
        info = text_generator.generate(state, 'recipe for', MAX_VAL_TOKENS, 1.0)
        print('\n')

<ipython-input-26-6916be85349e>:8: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  state = train_step(state, batch)


Epoch: 1	epoch time 0.21 min
	train loss: 4.0266, valid loss: 2.9790
Epoch: 2	epoch time 0.17 min
	train loss: 2.6842, valid loss: 2.4614
Epoch: 3	epoch time 0.18 min
	train loss: 2.3484, valid loss: 2.2507
Epoch: 4	epoch time 0.17 min
	train loss: 2.1758, valid loss: 2.1289
Epoch: 5	epoch time 0.17 min
	train loss: 2.0650, valid loss: 2.0471
Epoch: 6	epoch time 0.17 min
	train loss: 1.9834, valid loss: 1.9853
Epoch: 7	epoch time 0.18 min
	train loss: 1.9193, valid loss: 1.9347
Epoch: 8	epoch time 0.17 min
	train loss: 1.8673, valid loss: 1.8965
Epoch: 9	epoch time 0.18 min
	train loss: 1.8239, valid loss: 1.8637


KeyboardInterrupt: 

## 8. Generate texts

In [ ]:
# print prompt and top k candidate words probability
def print_probs(info, index_to_word, top_k=5):
    assert len(info) > 0, 'Please make `output_info=True`'
    for i in range(len(info)):
        start_tokens, word_probs = info[i].values()
        start_prompts = [index_to_word[token] for token in start_tokens if token != 0]
        start_prompts = ' '.join(start_prompts)
        print(f'\nPrompt: {start_prompts}')
        # word_probs
        probs_sorted = np.argsort(word_probs)[::-1][:top_k]
        for idx in probs_sorted:
            print(f'{index_to_word[idx]}\t{word_probs[idx] * 100:.2f}%')

In [ ]:
# Candidate words probability with temperature = 1.0
info = text_generator.generate(state, 
                               'recipe for roast', 
                               max_tokens=6, 
                               temperature=1.0, 
                               output_info=True)

print_probs(info, index_to_word, 5)

In [20]:
# Candidate words probability distribution with temperature = 1.0
info = text_generator.generate(state, 
                               'recipe for roast', 
                               max_tokens=6, 
                               temperature=0.2, 
                               output_info=True)

print_probs(info, index_to_word, 5)

recipe for roast until just tender

Prompt: recipe for roast until
until	96.50%
,	1.59%
in	0.90%
turkey	0.54%
chicken	0.43%

Prompt: recipe for roast until just
just	35.86%
smooth	24.25%
golden	17.78%
the	15.51%
mixture	2.17%

Prompt: recipe for roast until just tender
tender	68.73%
until	16.24%
cooked	14.73%
to	0.12%
before	0.12%


In [21]:
# generate text with temperature = 1.0
info = text_generator.generate(state, 
                               'recipe for roast', 
                               max_tokens=100, 
                               temperature=1.0, 
                               output_info=True)

recipe for roast them into 1 / 4 - poached chicken with garlic , tilting and put in a serrated knife . cover and ginger in 3 minutes . pour into 2 tablespoons fat remaining sauce warm water to 2 / 4 cup parsley . sprinkle with rice , and bring cherries soften to small bowl . do not crisp - mushroom mixture is smooth paste forms . broil for grilled eggplant is heated through , eggs until nuts are browned , and pepper . remove from heat and chopped fresh or until beginning to tear the beans in same


In [23]:
# generate text with temperature = 0.2
info = text_generator.generate(state, 
                               'recipe for roast', 
                               max_tokens=100, 
                               temperature=0.2, 
                               output_info=True)

recipe for roast until golden , and cook , about 5 minutes . add the mixture . add the mixture . add the garlic , about 5 minutes . add the mixture and cook , and pepper . transfer to a large bowl . add the mixture over medium heat , about 10 minutes . add the mixture to a large bowl . add the mixture to a large bowl . add the mixture , and refrigerate . add the mixture to a large bowl . add the mixture to a large bowl . add the bowl . add the
